In [30]:
import csv
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [31]:
#Create movie_url.csv 
link_df = pd.read_csv("links.csv")

In [55]:
link_df["imdbId"]

0        114709
1        113497
2        113228
3        114885
4        113041
         ...   
9737    5476944
9738    5914996
9739    6397426
9740    8391976
9741     101726
Name: imdbId, Length: 9742, dtype: int64

In [75]:
def generateLink(movie_id): 
    baseUrl = 'http://www.imdb.com/title/tt'
    ending = "/?ref_=fn_al_tt_1"
    if len(str(movie_id))==6: 
        movie_id = "0" + str(movie_id) 
    elif len(str(movie_id))==6: 
        movie_id = "00" + str(movie_id)
    else: 
        movie_id = str(movie_id)
    return baseUrl + movie_id + ending

def generateMovieId(movie_id): 
    baseUrl = 'tt'
    if len(str(movie_id))==6: 
        movie_id = "0" + str(movie_id) 

    elif len(str(movie_id))==5: 
        movie_id = "00" + str(movie_id)
    else: 
        movie_id = str(movie_id)
    return baseUrl + movie_id 

In [76]:
link_df["url"] = link_df.apply(lambda x: generateLink(x["imdbId"]), axis=1)
link_df["tconst"] = link_df.apply(lambda x: generateMovieId(x["imdbId"]), axis=1)

In [77]:
link_df.to_csv("movie_url.csv", index=False, header=True)

In [56]:
row_names = ['movieId','imdbId', 'tmdbId', 'url']
with open('movie_url.csv', 'r', newline='') as in_csv:
    reader = csv.DictReader(in_csv, fieldnames=row_names, delimiter=',')
    for row in reader:
        movie_id = row['movieId']
        movie_url = row['url']
        domain = 'http://www.imdb.com'
        with urllib.request.urlopen(movie_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            # Get url of poster image
            try:
                image_url = soup.find('div', class_='poster').a.img['src']
                # TODO: Replace hardcoded extension with extension from string itself
                extension = '.jpg'
                image_url = ''.join(image_url.partition('_')[0]) + extension
                filename = 'img/' + movie_id + extension
                print("test")
                print(filename)
                with urllib.request.urlopen(image_url) as response:
                    with open(filename, 'wb') as out_image:
                        out_image.write(response.read())
                    with open('movie_poster.csv', 'a', newline='') as out_csv:
                        writer = csv.writer(out_csv, delimiter=',')
                        writer.writerow([movie_id, image_url])
            # Ignore cases where no poster image is present
            except AttributeError:
                print("error")
        break

error


In [ ]:
row_names = ['movie_id', 'movie_url']
with open('movie_url.csv', 'r', newline='') as in_csv:
    reader = csv.DictReader(in_csv, fieldnames=row_names, delimiter=',')
    for row in reader:
        movie_id = row['movie_id']
        movie_url = row['movie_url']
        domain = 'http://www.imdb.com'
        with urllib.request.urlopen(movie_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            # Get url of poster image
            try:
                image_url = soup.find('div', class_='poster').a.img['src']
                # TODO: Replace hardcoded extension with extension from string itself
                extension = '.jpg'
                image_url = ''.join(image_url.partition('_')[0]) + extension
                filename = 'img/' + movie_id + extension
                with urllib.request.urlopen(image_url) as response:
                    with open(filename, 'wb') as out_image:
                        out_image.write(response.read())
                    with open('movie_poster.csv', 'a', newline='') as out_csv:
                        writer = csv.writer(out_csv, delimiter=',')
                        writer.writerow([movie_id, image_url])
            # Ignore cases where no poster image is present
            except AttributeError:
                pass

## Second Try 
https://github.com/tomkeith/imdb-scraper/blob/master/imdb-scraper.ipynb


In [3]:
import pandas as pd
import numpy as np

import csv
import sys

import requests
from bs4 import BeautifulSoup
    
from PIL import Image
from io import BytesIO

import re
import json

import time
import random
from pprint import pprint

In [26]:
def to_numeric(string, num_type='int'):
    '''
    Function to strip all non-numeric characters from string and return int or float
    INPUT - String to convert
          - num_type: either 'int' or 'float'
    OUTPUT - int or float type (returns original string if neither specified)
    '''
    if num_type == 'float':
        x = float( re.sub("[^0-9]", "", string ) )
    elif num_type == 'int':
        x = int( re.sub("[^0-9]", "", string ) )
    else:
        x = string
    return x


def savePoster(imdb_id, img_url):
    '''
    Function that fetches and save the poster image from provided url
    and saves it with the provided id (corresponding with IMDb).
    Won't replace (or even fetch) if file already exists.
    
    INPUT:  id from imdb, url where to find image
    OUTPUT: boolean flag if saved or not.
    '''
    import os.path
    
    # Get file extension
    ext = img_url.split('.')[-1]
    
    # Check to see if I already have it
    if os.path.isfile(f'posters/{imdb_id}.{ext}'):
        return False
    
    # Get image data, and save it as imdb_id
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))    
    img.save(f'posters/{imdb_id}.{ext}')
    
    return True

def concatenate_list_data(my_list):
    result = ''
    for element in my_list:
        result += str(element)
    return result

def time_since(start_time):
    '''
    Simple timer calculating time difference between
    start_time input parameter, and now
    
    OUTPUT: string ' 2m45s'
    INPUT: timestamp of starting time
    '''
    end_time = time.time()
    mins = (end_time - start_time)//60
    secs = (end_time - start_time) - (60*mins)
    return f'{mins:2.0f}m{secs:2.0f}s'

In [27]:
def imdb_scrape(imdb_id,movie_id,  save_image=False, debug=False):
    '''
    Function which scrapes IMDb using IMDb ID 'tt0107290'. Second parameter is for 
    the movie poster (saved in /posters/ folder). Third parameter is to print result.
    
    This function is mean to be used in a loop. As such, the print outputs may lack
    meaning if used outside of the cells below.
    
    INPUT:  - ID of movie to scrape from IMDB e.g. "tt0076759"
            - boolean to save the movie poster or not (default True)
            - boolean to print result
           
    OUTPUT: Dictionary of various scrapped information.
    
             {'tconst':imdb_id, 'title':'',     'release_year':'',     'release_date':'',
              'MPAA':'',        'genre':[],     'runtime':'',          'poster_url':'',
              'plot_short':'',  'plot_long':'', 'imdb_rating':'',      'num_imdb_votes':'',
              'metacritic':'',  'num_user_reviews':'',                 'num_critic_reviews':''
             }
    '''
    # Target datapoints to scrape (with provided imdb_id)
    imdb_info_dict = {'movieid':movie_id, 'tconst':imdb_id,'title':'',  
                      'MPAA':'',       'genre':[],            'poster_url':'',
                      'description':'','reviewBody':'','keywords': '', 'imdb_rating':'',      'num_imdb_votes':'',
                     }
    imdb_info_dict['tconst'] = imdb_id
    
    imdb_base_url = 'https://www.imdb.com/title/'
    print(f'{imdb_id.ljust(10)} ', end='')
    
    # Main content - build URL, and soup content
    imdb_full_url = imdb_base_url + imdb_id
    r = requests.get(imdb_full_url).content
    soup = BeautifulSoup(r, 'html.parser')
    print(f'[x]   ', end='')
    
    # Code from js section has json variables
    json_dict = json.loads( str( soup.findAll('script', {'type':'application/ld+json'})[0].text ))

    # Info - Movie title, year, parental content rating, poster url
    imdb_info_dict['title'] = json_dict['name']
    if 'contentRating' in json_dict:
        imdb_info_dict['MPAA'] = json_dict['contentRating'] 
    imdb_info_dict['poster_url'] = json_dict['image']
    imdb_info_dict['description'] = json_dict['description']
    imdb_info_dict['keywords'] = json_dict['keywords']

    # imdb_info_dict['release_year'] = int( soup.find('span', {'id':'titleYear'}).a.text )
    # imdb_info_dict['runtime'] = to_numeric( soup.find('time')['datetime'] )


    # Release date (from top header)
    # date_string = soup.find('div', {'class':'title_wrapper'}).findAll('a')[-1].text.split(' (')[0]
    # imdb_info_dict['release_date'] = date_string
    
    # Genres (up to 7)
    imdb_info_dict['genre'] = json_dict['genre']

    # Ratings - IMDb rating (and vote count), Metacritic
    imdb_info_dict['imdb_rating'] = float( json_dict['aggregateRating']['ratingValue'] )
    imdb_info_dict['num_imdb_votes'] = json_dict['aggregateRating']['ratingCount']

    # Metacritic score, if there is one
    # if soup.find('div', {'class':'metacriticScore'}) != None:
    #     imdb_info_dict['metacritic'] = int( soup.find('div', {'class':'metacriticScore'}).span.text )

    # Reviews - Number of critic and public reviews (different than ratings/votes)
    num_review_list = soup.findAll('div',{'class':'titleReviewBarItem titleReviewbarItemBorder'})
    if num_review_list != []:
        reviews = num_review_list[0].findAll('a')
        if len(reviews) > 1:
            imdb_info_dict['num_critic_reviews'] = to_numeric( reviews[1].text )
        if len(reviews) > 0:
            imdb_info_dict['num_user_reviews'] = to_numeric( reviews[0].text )

    # Plots - long and short versions
    # imdb_info_dict['plot_short'] = soup.find('div',{'class':'summary_text'}).text.strip()
    # if 'Add a Plot' in imdb_info_dict['plot_short']:
    #     imdb_info_dict['plot_short'] = ''
    # if soup.find('div',{'id':'titleStoryLine'}).div.p != None:
    #     imdb_info_dict['plot_long'] = soup.find('div',{'id':'titleStoryLine'}).div.p.span.text.strip()
    
    # Plot output
    print(f'[x]   ', end='')

    if save_image == True:
        img_status = savePoster(imdb_id, imdb_info_dict['poster_url'])
        if img_status == True:
            print(f'[x]   ', end='')
        else:
            print(f'[ ]   ', end='')
    else:
        print(f'N/A   ', end='')
    
    print(f"{(imdb_info_dict['title']+' ('')')[:100]:100} ", end='')
    # time.sleep(random.randint(1,10) / 
    
    print('')
    if(debug):
        pprint(imdb_info_dict)
    return imdb_info_dict

In [28]:
import csv
import sys



def scrapeMovies(startInterval, endInterval, movie_df): 
    # xNeed to provide a csv of IMDb IDs to scrape

    failed_list = []


    print('--------------------------------------------------------------------------')    
    print('')
    print('Count   tconst     Get   Parse Img   Title')

    start_time = time.time()
    annual_movie_list = []
    fails=0
    for i, tconst in enumerate(movie_df.iloc[startInterval:endInterval,:]['tconst'].values):
        print(f'{i+1:5d}   ', end = '')
        try:
            movie_id =str(int(movie_df[movie_df["tconst"]==tconst]["movieId"]))
            scraped_movie_info = imdb_scrape(tconst, movie_id, False) #Change to false to NOT save movie poster
            annual_movie_list.append(scraped_movie_info)
        except Exception:
            print(f'--------- FAILED ---------- FAILED ---------- FAILED ----------')
            failed_list.append(tconst)
            fails+=1

    print(f'Movies scraped: {len(annual_movie_list)}   Fails: {fails}   ', end='')
    my_df = pd.DataFrame(annual_movie_list)

    #Load old df 
    oldDf = pd.read_csv('imdb_scrape.csv', delimiter='\t')
    newDF = my_df.append(oldDf, ignore_index=True)
    newDF.to_csv(f'imdb_scrape.csv', sep='\t', quoting=csv.QUOTE_ALL, index=False)
    print('\n')
    print(f'Saved: imdb_scrape.csv     ', end='')
    print('')

In [32]:
movie_df = pd.read_csv('movie_url.csv')
oldDf = pd.read_csv('imdb_scrape.csv', delimiter='\t')

In [33]:
# 1. Get all the movie_ids 
alreadyLoadedDF = set(oldDf["movieid"].values)

In [61]:
type(alreadyLoadedDF)

set

In [34]:
movie_df_filtered = movie_df[movie_df["movieId"].isin(alreadyLoadedDF)==False]

In [36]:
movie_df_filtered.shape

(5286, 5)

In [17]:
movie_df[movie_df["movieId"]==193579]


,movieId,imdbId,tmdbId,url,tconst
9736,193579,5342766,360617.0,http://www.imdb.com/title/tt5342766/?ref_=fn_a...,tt5342766


In [37]:
max = 5286
current_progress = 0
iteration = list(np.arange(current_progress,5286, 10))
lastElement = current_progress +1
for currentElement in iteration: 
    scrapeMovies(lastElement, currentElement, movie_df_filtered)
    lastElement = currentElement
    


scrapeMovies(lastElement, max, movie_df_filtered)

--------------------------------------------------------------------------

Count   tconst     Get   Parse Img   Title
Movies scraped: 0   Fails: 0   

Saved: imdb_scrape.csv     
--------------------------------------------------------------------------

Count   tconst     Get   Parse Img   Title
    1   tt0114709  [x]   [x]   N/A   Toy Story ()                                                                                         
    2   tt0113497  [x]   [x]   N/A   Jumanji ()                                                                                           
    3   tt0113228  [x]   [x]   N/A   Grumpier Old Men ()                                                                                  
    4   tt0114885  [x]   [x]   N/A   Waiting to Exhale ()                                                                                 
    5   tt0113041  [x]   [x]   N/A   Father of the Bride Part II ()                                                                       
    6 

In [18]:
list(np.arange(0,9743, 100))

[0,
 100,
 200,
 300,
 400,
 500,
 600,
 700,
 800,
 900,
 1000,
 1100,
 1200,
 1300,
 1400,
 1500,
 1600,
 1700,
 1800,
 1900,
 2000,
 2100,
 2200,
 2300,
 2400,
 2500,
 2600,
 2700,
 2800,
 2900,
 3000,
 3100,
 3200,
 3300,
 3400,
 3500,
 3600,
 3700,
 3800,
 3900,
 4000,
 4100,
 4200,
 4300,
 4400,
 4500,
 4600,
 4700,
 4800,
 4900,
 5000,
 5100,
 5200,
 5300,
 5400,
 5500,
 5600,
 5700,
 5800,
 5900,
 6000,
 6100,
 6200,
 6300,
 6400,
 6500,
 6600,
 6700,
 6800,
 6900,
 7000,
 7100,
 7200,
 7300,
 7400,
 7500,
 7600,
 7700,
 7800,
 7900,
 8000,
 8100,
 8200,
 8300,
 8400,
 8500,
 8600,
 8700,
 8800,
 8900,
 9000,
 9100,
 9200,
 9300,
 9400,
 9500,
 9600,
 9700]

In [5]:
import csv
import sys

# xNeed to provide a csv of IMDb IDs to scrape
movie_df = pd.read_csv('movie_url.csv')

failed_list = []


print('--------------------------------------------------------------------------')    
print('')
print('Count   tconst     Get   Parse Img   Title')

start_time = time.time()
annual_movie_list = []
fails=0

# for i, tconst in enumerate(movie_df.iloc[0:5, :]['tconst'].values):
for i, tconst in enumerate(movie_df['tconst'].values):
    print(f'{i+1:5d}   ', end = '')
    try:
        scraped_movie_info = imdb_scrape(tconst, False) #Change to false to NOT save movie poster
        annual_movie_list.append(scraped_movie_info)
    except Exception:
        print(f'--------- FAILED ---------- FAILED ---------- FAILED ----------')
        failed_list.append(tconst)
        fails+=1

print(f'Movies scraped: {len(annual_movie_list)}   Fails: {fails}   ', end='')

my_df = pd.DataFrame(annual_movie_list)
my_df.to_csv(f'imdb_scrape.csv', sep='\t', quoting=csv.QUOTE_ALL, index=False)
print('\n')
print(f'Saved: imdb_scrape.csv     ', end='')
print(f'Time taken: {time_since(start_time)}')
print('')

--------------------------------------------------------------------------

Count   tconst     Get   Parse Img   Title
    1   tt0114709  [x]   [x]   N/A   Toy Story ()                                                                                         
    2   tt0113497  [x]   [x]   N/A   Jumanji ()                                                                                           
    3   tt0113228  [x]   [x]   N/A   Grumpier Old Men ()                                                                                  
    4   tt0114885  [x]   [x]   N/A   Waiting to Exhale ()                                                                                 
    5   tt0113041  [x]   [x]   N/A   Father of the Bride Part II ()                                                                       
    6   tt0113277  [x]   [x]   N/A   Heat ()                                                                                              
    7   tt0114319  [x]   [x]   N/A   Sabrina ()

KeyboardInterrupt: 